下面是完整搜索过程的代码，包括：

1. 试验 500 个问题中有多少问题在两个向量数据库中的密集检索结果是不一样的

    检索结果一样的定义为：

    - 检索分数一样
    - 检索到的文档内容一样

2. 利用问题「ICI House is now named after the company that provides what type of item?」的检索结果重新构建两个向量数据库，看是否能够复现出问题

### 看 500 个问题中是否有更多的检索结果不同的情况

上面的代码证明了两个数据库中所存储的实体是完全相同的

接下来，我们就看一下，如果将这 500 个问题分别在这两个向量数据库中进行检索，检索得到不一致文档的问题有多少

具体的逻辑为：

1. 前期准备

    加载 500 个问题、加载嵌入模型、加载两个向量数据库、定义判断检索结果是否一致的函数

2. 遍历 500 个问题，将检索结果不一致的问题和结果记录下来

In [ ]:
# 加载 500 个问题
import json
from loguru import logger as log
import pprint

all_questions = []
question_path = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\evaluation\hotpotqa\hybrid_test_data_500.json'

with open(question_path, 'r', encoding='utf-8') as f:
    all_questions = json.load(f)

log.info(f'加载了 {len(all_questions)} 个问题')

pprint.pprint(all_questions[0])

2025-07-10 14:50:13.178 | INFO     | __main__:<module>:12 - 加载了 500 个问题


{'_id': '5a8ba761554299240d9c2066',
 'answer': 'nearly 80 years',
 'context': [['Bus Stop (TV series)',
              ['Bus Stop is a 26-episode American drama which aired on ABC '
               'from October 1, 1961, until March 25, 1962, starring Marilyn '
               'Maxwell as Grace Sherwood, the owner of a bus station and '
               'diner in the fictitious town of Sunrise in the Colorado '
               'Rockies.',
               ' The program was adapted from William Inge\'s play, "Bus '
               'Stop", and Inge was a script consultant for the series, which '
               'followed the lives of travelers passing through the bus '
               'station and the diner.',
               " Maxwell's co-stars were Richard Anderson as District Attorney "
               'Glenn Wagner, Rhodes Reason as Sheriff Will Mayberry, Joan '
               'Freeman as waitress Elma Gahrigner, Bernard Kates as Ralph the '
               'coroner, and Buddy Ebsen as Virge Bles

In [ ]:
# 定义嵌入模型
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embed_model = OpenAIEmbeddings(
    api_key=os.getenv("BL_API_KEY"),
    base_url=os.getenv("BL_BASE_URL"),
    model="text-embedding-v3",
    dimensions=1024,
    check_embedding_ctx_length=False
)

In [ ]:
# 加载两个向量数据库
from langchain_milvus import Milvus, BM25BuiltInFunction

dense_vs = Milvus(
    collection_name='dense_hotpotqa_500',
    embedding_function=embed_model,
)

hybrid_vs = Milvus(
    collection_name='hybrid_hotpotqa_500',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)

In [ ]:
# 定义判断检索结果是否一致的函数
from typing import Tuple, List
from langchain_core.documents import Document

def is_same_result(
        dense_res: List[Tuple[Document, float]],
        hybrid_res: List[Tuple[Document, float]]
) -> bool:
    """
    比较两个向量数据库进行密集检索的结果是否一致

    Args:
        dense_res (List[Tuple[Document, float]]): 密集检索的结果
        hybrid_res (List[Tuple[Document, float]]): 混合检索的结果

    Returns:
        bool: 检索结果是否一致
    """
    # 检查两个结果的数量是否一致
    if len(dense_res) != len(hybrid_res):
        return False

    # 检查两个结果里面的具体内容是否一样
    for d_res, h_res in zip(dense_res, hybrid_res):
        if (
            d_res[1] != h_res[1]  # 比较分数
            or d_res[0].page_content != h_res[0].page_content  # 比较文档内容
        ):
            return False
        
        # 由于两个向量数据库的字段有些不同，需要做些修改才能直接比较
        # 1. 将 dense_res 中的 'vector' 字段改为 'dense'  ->  my_dict['new_key'] = my_dict.pop('old_key')
        # 2. 不输出 hybrid_res 中的 'sparse' 字段
        # 3. 不比较两个检索结果中的 pk 字段

        # 第 2 步将在混合嵌入向量数据库检索时通过限制 output_fields 来实现

        # d_res[0].metadata['dense'] = d_res[0].metadata.pop('vector')
        # d_res[0].metadata.pop('pk')
        # h_res[0].metadata.pop('pk')

        # if d_res[0].metadata != h_res[0].metadata:
        #     return False

        # 比较 metadata 似乎没有意义

    return True

In [ ]:
# 在两个向量数据库上进行检索
from tqdm import tqdm

diff_res = []
same_res_num = 0

hybrid_output_fields = hybrid_vs.fields[:]
hybrid_output_fields.remove('sparse')

for question in tqdm(all_questions):
    dense_res = dense_vs.similarity_search_with_score(
        query=question['question'],
        k=10
    )

    raw_res = hybrid_vs.client.search(
        collection_name='hybrid_hotpotqa_500',
        data=[embed_model.embed_query(question['question'])],
        anns_field='dense',
        limit=10,
        search_params={
            'metric_type': 'L2',
            'params': {}
        },
        output_fields=hybrid_output_fields
    )

    hybrid_res = hybrid_vs._parse_documents_from_search_results(raw_res)

    # 比较检索结果
    if is_same_result(dense_res, hybrid_res):
        same_res_num += 1
    else:
        tmp = {
            'question': question['question'],
            'dense_res': dense_res,
            'hybrid_res': hybrid_res
        }
        diff_res.append(tmp)

        # log.info(f'{question["question"]} 检索结果不一致')
        tqdm.write(f'{question["question"]} 检索结果不一致')
# 输出结果
log.info(f'检索结果一致的问题数量：{same_res_num} / {len(all_questions)}')
log.info(f'检索结果不一致的问题数量：{len(diff_res)}')

  0%|          | 1/500 [00:00<02:27,  3.38it/s]

How long did the career span of the actor who starred with Mickey Rooney and Marilyn Maxwell in Off Limits? 检索结果不一致


  2%|▏         | 10/500 [00:03<02:34,  3.16it/s]

Who was the running mate of the man about whose candidacy the 2016 book Insane Clown President: Dispatches from the 2016 Circus documents? 检索结果不一致


  3%|▎         | 15/500 [00:04<02:12,  3.65it/s]

...

 99%|█████████▊| 493/500 [02:31<00:01,  3.74it/s]

What is the estimated population of the city in which the suburb of Beaumont is located ? 检索结果不一致


100%|█████████▉| 499/500 [02:33<00:00,  2.75it/s]

From what county is Steven Avery, wrongfully convicted man on which the documentary Making a Murderer is based?  检索结果不一致


100%|██████████| 500/500 [02:33<00:00,  3.25it/s]
2025-07-10 15:31:23.829 | INFO     | __main__:<module>:44 - 检索结果一致的问题数量：314 / 500
2025-07-10 15:31:23.830 | INFO     | __main__:<module>:45 - 检索结果不一致的问题数量：186


In [ ]:
# 把检索不一致的内容持久化到文件中
res = []

for diff in diff_res:
    tmp = {
        'question': diff['question'],
    }

    d_res = diff['dense_res']
    h_res = diff['hybrid_res']

    i = 0
    for d_r, h_r in zip(d_res, h_res):
        tmp[str(i)] = {
            'dense_score': d_r[1],
            'hybrid_score': h_r[1],
            'dense_content': d_r[0].page_content,
            'hybrid_content': h_r[0].page_content
        }
        i += 1
    
    res.append(tmp)


res_path = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\diff.json'
os.makedirs(os.path.dirname(res_path), exist_ok=True)

with open(res_path, 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False)

In [ ]:
# 统计 500 个问题中，top_1 到 top_10 中不一样的分别有多少
from collections import defaultdict

res = defaultdict(int)
less_3 = 0

for diff in diff_res:
    d_res = diff['dense_res']
    h_res = diff['hybrid_res']

    i = 0
    for d_r, h_r in zip(d_res, h_res):
        if d_r[1] != h_r[1]:
            res[str(i)] += 1
            if i < 3:
                log.info(f'{i}: {diff['question']}')
                less_3 += 1
            break
        i += 1

2025-07-10 16:06:21.010 | INFO     | __main__:<module>:15 - 2: Spider9 was founded in 2011 by the head of which subsidiary of Wanxiang Group?
2025-07-10 16:06:21.010 | INFO     | __main__:<module>:15 - 0: ICI House is now named after the company that provides what type of item?
2025-07-10 16:06:21.011 | INFO     | __main__:<module>:15 - 2: The musician who did "Khalaara" was born on what date?
2025-07-10 16:06:21.011 | INFO     | __main__:<module>:15 - 2: When was the younger brother of Ervin Randle born?
2025-07-10 16:06:21.012 | INFO     | __main__:<module>:15 - 1: Swingin' Down Yonder is the first full-length, 12-inch album recorded by a singer known by what nickname?
2025-07-10 16:06:21.013 | INFO     | __main__:<module>:15 - 0: How many total staff (including part-time, flexible, and fixed contract staff) does the broadcaster, who released HyperNormalisation, employ?
2025-07-10 16:06:21.013 | INFO     | __main__:<module>:15 - 2: Are Harry Everett Smith and Vladimir Danilevich both

In [ ]:
res

defaultdict(int,
            {'4': 22,
             '9': 33,
             '8': 27,
             '2': 11,
             '7': 19,
             '5': 16,
             '0': 12,
             '3': 13,
             '6': 23,
             '1': 10})

In [ ]:
less_3

33

In [ ]:
# 记录 top_1 不同的问题和检索到的结果和分数
top1_diff = []

for diff in diff_res:
    d_res = diff['dense_res']
    h_res = diff['hybrid_res']

    i = 0
    for d_r, h_r in zip(d_res, h_res):
        if i == 0 and d_r[1] != h_r[1]:
            log.info(f'{i}: {diff["question"]}')
            top1_diff.append(diff)
            break
        i += 1

2025-07-10 17:00:27.508 | INFO     | __main__:<module>:10 - 0: ICI House is now named after the company that provides what type of item?
2025-07-10 17:00:27.509 | INFO     | __main__:<module>:10 - 0: How many total staff (including part-time, flexible, and fixed contract staff) does the broadcaster, who released HyperNormalisation, employ?
2025-07-10 17:00:27.510 | INFO     | __main__:<module>:10 - 0:  What species has an extinct subspecies that are classified as Trogloxene and are known from fossils and prehistoric art?
2025-07-10 17:00:27.510 | INFO     | __main__:<module>:10 - 0: What company did Roscoe Channing form with the owner of a leading stable of thoroughbred racehorses?
2025-07-10 17:00:27.511 | INFO     | __main__:<module>:10 - 0: Who was older, Andrew Preston or James Taylor?
2025-07-10 17:00:27.511 | INFO     | __main__:<module>:10 - 0: Kenneth L. Gile is the Chief Operating Officer of an airline with its head office in what airport?
2025-07-10 17:00:27.512 | INFO     | 

In [ ]:
len(top1_diff)

12

In [ ]:
show = []

for diff in top1_diff:
    d_res = diff['dense_res'][0]
    h_res = diff['hybrid_res'][0]

    tmp = {
        'question': diff['question'],
        'dense_top1': d_res[1],
        'hybrid_top1': h_res[1],
        'dense_content': d_res[0].page_content,
        'hybrid_content': h_res[0].page_content
    }

    show.append(tmp)
show
# 把检索不一致的内容持久化到文件中
res_path = r'C:\Users\Lenovo\Desktop\RAG rebuild v2\top1_diff.json'
os.makedirs(os.path.dirname(res_path), exist_ok=True)

with open(res_path, 'w', encoding='utf-8') as f:
    json.dump(show, f, ensure_ascii=False)

### 尝试复现问题

In [ ]:
# 将第一个 top_1 不同问题 两个向量数据库检索到的内容都输出出来
# 构建一个类似的向量数据库，看能否复现出来类似的问题
d_res = top1_diff[0]['dense_res']
h_res = top1_diff[0]['hybrid_res']

for d_r, h_r in zip(d_res, h_res):
    print('-' * 50)
    print(f'dense_score: \n{d_r[1]}\n')
    print(f'hybrid_score: \n{h_r[1]}\n')
    print(f'dense_content: \n{d_r[0].page_content}\n')
    print(f'hybrid_content: \n{h_r[0].page_content}\n')
    print()

--------------------------------------------------
dense_score: 
0.7522499561309814

hybrid_score: 
0.9679315090179443

dense_content: 
ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings

hybrid_content: 
, then a record among magazines published by Time, Inc., SI's parent company.


--------------------------------------------------
dense_score: 
0.8594906330108643

hybrid_score: 
0.9679315090179443

dense_content: 
ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its sh

In [ ]:
# 定义嵌入模型
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

load_dotenv()

embed_model = OpenAIEmbeddings(
    api_key=os.getenv("BL_API_KEY"),
    base_url=os.getenv("BL_BASE_URL"),
    model="text-embedding-v3",
    dimensions=1024,
    check_embedding_ctx_length=False
)

In [ ]:
# 定义要存储的文档
from langchain_core.documents import Document

docs = [
    Document(
        page_content='ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne\'s long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings',
        metadata={
            'id': 'dense_1',
            'score': 0.7522499561309814
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'hybrid_1',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content='ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain\'s defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated.',
        metadata={
            'id': 'dense_2',
            'score': 0.8594906330108643
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'hybrid_2',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'dense_3',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content='Villard, also known as Villard Books, is a publishing imprint of Random House, one of the largest publishing companies in the world. It was founded in 1983. Villard began as an independent imprint of Random House and is currently a sub-imprint of Ballantine Books, itself an imprint of Random House. It was named after a Stanford White brownstone mansion on Madison Avenue that was the home of Random House for twenty years.',
        metadata={
            'id': 'hybrid_3',
            'score': 0.989403247833252
        }
    ),
    Document(
        page_content=', then a record among magazines published by Time, Inc., SI\'s parent company.',
        metadata={
            'id': 'dense_4',
            'score': 0.9679315090179443
        }
    ),
    Document(
        page_content=', Illinois, and they have an annual catalog circulation exceeding 50 million. The company is employee owned and considered to be a renowned purveyor of gadgetry and elegant gifts. Every item that is sold comes with the Hammacher Schlemmer Lifetime Guarantee ("We make an unconditional and unwavering promise: Our merchandise is guaranteed for life")',
        metadata={
            'id': 'hybrid_4',
            'score': 0.9983643293380737
        }
    ),
    Document(
        page_content='Izze (pronounced iz-ee) is the brand name of a line of carbonated juice drinks produced by the IZZE Beverage Company in Boulder, Colorado, which is owned by PepsiCo. The drinks consist of 70% fruit juice from concentrate, and 30% seltzer water. Izze is an all-natural, no-preservatives-added fruit soda.',
        metadata={
            'id': 'dense_5',
            'score': 0.9726308584213257
        }
    ),
    Document(
        page_content='Dodge Correctional Institution, (DCI), is an adult male maximum-security correctional facility operated by the Wisconsin Department of Corrections Division of Adult Institutions in Waupun, Wisconsin. The facility was converted from the Central State Hospital for the Criminally Insane to an adult correctional facility in 1977 at a cost of $2.47 million of general obligation bonds',
        metadata={
            'id': 'hybrid_5',
            'score': 1.0290902853012085
        }
    ),
    Document(
        page_content='The Tongil Industries Company Co., Ltd., (in short the “TIC”), is a South Korean heavy industry company headquartered in Changwon City, South Korea. TIC was founded in July, 1988 originally as the Jin Heung Machinery Co., Ltd. As of 2011, it comprises 4 business divisions; Machine tools, Ball Screws, Automobile Components and Heat Treatment. The Tongil Industries is a subsidiary of the TONGIL Group, a South Korean business conglomerate (chaebol) managed by Kook Jin “Justin” Moon',
        metadata={
            'id': 'dense_6',
            'score': 0.9825005531311035
        }
    ),
    Document(
        page_content=', yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff',
        metadata={
            'id': 'hybrid_6',
            'score': 1.0295854806900024
        }
    ),
    Document(
        page_content=', to Morse International. Morse filed for bankruptcy and Diving Equipment and Supply Company (DESCO) acquired its assets in 2016. DESCO reverted to the name A J Morse & Son and Morse products will be marketed under that name. DESCO\'s business plan is to bring back the quality and products associated with the earlier name. DESCO has on re-introduced the breast plate feed (air being fed into the breast plate rather than the bonnet)helmet design from the early 1900s as its first offering. They also',
        metadata={
            'id': 'dense_7',
            'score': 0.9825825691223145
        }
    ),
    Document(
        page_content=', yet the crash created "Watch Dog" like HAMP TEAM, IRS, COMPTROLLER< Federal Trade Commission Consumer Protection Bureau, FBI, CIA, Local Police Department, ICE ( The FBI online Computer crime division receives and investigates computer crimes that record keeping staff from title companies, lending institutional staff',
        metadata={
            'id': 'hybrid_7',
            'score': 1.0295854806900024
        }
    ),
    Document(
        page_content='Desilu Productions ( ) was an American production company founded and co-owned by husband and wife Desi Arnaz and Lucille Ball, best known for shows such as "I Love Lucy", "", and "The Untouchables". Until 1962, Desilu was the second-largest independent television production company in the U.S. behind MCA\'s Revue Productions until MCA bought Universal Pictures',
        metadata={
            'id': 'dense_8',
            'score': 0.9858689904212952
        }
    ),
    Document(
        page_content='Coty, Inc., is a North American beauty products manufacturer based in New York founded in Paris, France, by François Coty in 1904. Its main products are fragrances, colour cosmetics and skin and body care products. It is known for its cooperation with designers and celebrities for the creation of fragrances. Its biggest brands, or "power brands" as it calls them, are: Calvin Klein (fragrance and cosmetics), Chloe (fragrance), Davidoff (fragrance), y (fragrance), Marc Jacobs (fragrance)',
        metadata={
            'id': 'hybrid_8',
            'score': 1.0300822257995605
        }
    ),
    Document(
        page_content='Villard, also known as Villard Books, is a publishing imprint of Random House, one of the largest publishing companies in the world. It was founded in 1983. Villard began as an independent imprint of Random House and is currently a sub-imprint of Ballantine Books, itself an imprint of Random House. It was named after a Stanford White brownstone mansion on Madison Avenue that was the home of Random House for twenty years.',
        metadata={
            'id': 'dense_9',
            'score': 0.989403247833252
        }
    ),
    Document(
        page_content=', clothing, and fragrances being released.',
        metadata={
            'id': 'hybrid_9',
            'score': 1.0305763483047485
        }
    ),
    Document(
        page_content=', Illinois, and they have an annual catalog circulation exceeding 50 million. The company is employee owned and considered to be a renowned purveyor of gadgetry and elegant gifts. Every item that is sold comes with the Hammacher Schlemmer Lifetime Guarantee ("We make an unconditional and unwavering promise: Our merchandise is guaranteed for life").',
        metadata={
            'id': 'dense_10',
            'score': 0.9983643293380737
        }
    ),
    Document(
        page_content=', as well as catering items such as sandwich trays and boxed lunches. The chain is also known for its McAlister\'s Famous Sweet Tea, which is available by the glass or by the gallon.',
        metadata={
            'id': 'hybrid_10',
            'score': 1.0356755256652832
        }
    )

]

# 为其分配 id 
ids = [str(i + 1) for i in range(len(docs))]

In [ ]:
# 使用 langchain_milvus 定义「仅包含密集嵌入的向量数据库」和「包含稀疏嵌入和密集嵌入的向量数据库」
from langchain_milvus import Milvus, BM25BuiltInFunction

# 仅包含密集嵌入的向量数据库
dense_vs = Milvus(
    collection_name='dense_only',
    embedding_function=embed_model,
    vector_field='dense'
)
# 向其中添加实体
dense_vs.add_documents(documents=docs[:10], ids=ids[:10])  # 嵌入模型一次无法接受太多文档7])  # 嵌入模型一次无法接受太多文档
dense_vs.add_documents(documents=docs[10:], ids=ids[10:])


# 包含稀疏嵌入和密集嵌入的向量数据库
hybrid_vs = Milvus(
    collection_name='dense_and_sparse',
    embedding_function=embed_model,
    builtin_function=BM25BuiltInFunction(output_field_names='sparse'),
    vector_field=['dense', 'sparse']
)
# 向其中添加实体
hybrid_vs.add_documents(documents=docs[:10], ids=ids[:10])
hybrid_vs.add_documents(documents=docs[10:], ids=ids[10:])

['11', '12', '13', '14', '15', '16', '17', '18', '19', '20']

In [ ]:
# 检索
query = 'ICI House is now named after the company that provides what type of item?'

dense_res = dense_vs.similarity_search_with_score(
    query=query,
    k=10
)

hybrid_output_fields = hybrid_vs.fields[:]
hybrid_output_fields.remove('sparse')

raw_hybrid_res = hybrid_vs.client.search(
    collection_name='dense_and_sparse',
    data=[embed_model.embed_query(query)],
    anns_field='dense',
    limit=10,
    search_params={
        'metric_type': 'L2',
        'params': {}
    },
    output_fields=hybrid_output_fields
)
hybrid_res = hybrid_vs._parse_documents_from_search_results(raw_hybrid_res)

In [ ]:
import pprint

pprint.pprint(dense_res)

[(Document(metadata={'pk': '1', 'id': 'dense_1', 'score': 0.7522499561309814}, page_content="ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings"),
  0.7522499561309814),
 (Document(metadata={'pk': '3', 'id': 'dense_2', 'score': 0.8594906330108643}, page_content="ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain's defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated."),


In [ ]:
pprint.pprint(hybrid_res)

[(Document(metadata={'score': 0.7522499561309814, 'pk': '1', 'id': 'dense_1'}, page_content="ICI House (now Orica House) is a 19-storey office building in East Melbourne, Victoria, Australia. Begun in 1955, it was the tallest building in Australia upon completion in 1958, breaking Melbourne's long standing 132ft height limit, and was the first International Style skyscraper in the country. It symbolised progress, modernity, efficiency and corporate power in postwar Melbourne, and heralded the construction of the high-rise office buildings"),
  0.7522499561309814),
 (Document(metadata={'score': 0.8594906330108643, 'pk': '3', 'id': 'dense_2'}, page_content="ICI Fibres developed the Crimplene fibre. It is a thick, polyester yarn used to make a fabric of the same name. The resulting cloth is heavy, wrinkle-resistant and retains its shape well. Britain's defunct ICI Laboratory developed the fibre in the early 1950s and named it after the Crimple Valley in which the company was situated."),
